# Developing CRF model for Named Entity Recognition 

* geo = Geographical Entity
* org = Organization
* per = Person
* gpe = Geopolitical Entity
* tim = Time indicator
* art = Artifact
* eve = Event
* nat = Natural Phenomenon

#### Importing Libraries

In [42]:

from sklearn_crfsuite.metrics import flat_f1_score, flat_classification_report, flat_precision_score, flat_recall_score, flat_accuracy_score, sequence_accuracy_score
from sklearn.model_selection import train_test_split
from sklearn_crfsuite import CRF
import pandas as pd

In [43]:
#Reading the csv file
df = pd.read_csv('ner_dataset.csv', encoding = "ISO-8859-1")

In [44]:
df[df['Tag'] != 'O'].head(20)

,Sentence #,Word,POS,Tag
6,NaN,London,NNP,B-geo
12,NaN,Iraq,NNP,B-geo
18,NaN,British,JJ,B-gpe
42,NaN,Bush,NNP,B-per
65,NaN,Hyde,NNP,B-geo
66,NaN,Park,NNP,I-geo
94,NaN,Britain,NNP,B-geo
97,NaN,Labor,NNP,B-org
98,NaN,Party,NNP,I-org
102,NaN,English,JJ,B-gpe


In [45]:
df.describe()

,Sentence #,Word,POS,Tag
count,47959,1048565,1048575,1048575
unique,47959,35177,42,17
top,Sentence: 1,the,NN,O
freq,1,52573,145807,887908


In [46]:
#Displaying the unique Tags
df['Tag'].unique()

array(['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim',
       'B-art', 'I-art', 'I-per', 'I-gpe', 'I-tim', 'B-nat', 'B-eve',
       'I-eve', 'I-nat'], dtype=object)

In [47]:
df

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
...,...,...,...,...
1048570,NaN,they,PRP,O
1048571,NaN,responded,VBD,O
1048572,NaN,to,TO,O
1048573,NaN,the,DT,O


In [48]:
#Checking null values, if any.
df.isnull().sum()

Sentence #    1000616
Word               10
POS                 0
Tag                 0
dtype: int64

There are lots of missing values in 'Sentence #' attribute. So we will use pandas fillna technique and use 'ffill' method which propagates last valid observation forward to next.

In [49]:
df = df.fillna(method = 'ffill')

C:\Users\pubg3\AppData\Local\Temp\ipykernel_16200\439241999.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method = 'ffill')


## Converting df to tuple to give input to --> crf model  

In [50]:
# This is a class te get sentence. The each sentence will be list of tuples with its tag and pos.
class sentence(object):
    def __init__(self, df):
        self.n_sent = 1
        self.df = df
        self.empty = False
        agg = lambda s : [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(),s['POS'].values.tolist(),s['Tag'].values.tolist())]
        self.grouped = self.df.groupby("Sentence #").apply(agg)
        self.sentences = [s for s in self.grouped]
        
    def get_text(self):
        try:
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent +=1
            return s
        except:
            return None

In [51]:
#Displaying one full sentence
#getter = sentence(df)
sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
sentences[0]

'Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .'

In [52]:
#sentence with its pos and tag.
sent = getter.get_text()
print(sent)
sentences = getter.sentences

[('Families', 'NNS', 'O'), ('of', 'IN', 'O'), ('soldiers', 'NNS', 'O'), ('killed', 'VBN', 'O'), ('in', 'IN', 'O'), ('the', 'DT', 'O'), ('conflict', 'NN', 'O'), ('joined', 'VBD', 'O'), ('the', 'DT', 'O'), ('protesters', 'NNS', 'O'), ('who', 'WP', 'O'), ('carried', 'VBD', 'O'), ('banners', 'NNS', 'O'), ('with', 'IN', 'O'), ('such', 'JJ', 'O'), ('slogans', 'NNS', 'O'), ('as', 'IN', 'O'), ('"', '``', 'O'), ('Bush', 'NNP', 'B-per'), ('Number', 'NN', 'O'), ('One', 'CD', 'O'), ('Terrorist', 'NN', 'O'), ('"', '``', 'O'), ('and', 'CC', 'O'), ('"', '``', 'O'), ('Stop', 'VB', 'O'), ('the', 'DT', 'O'), ('Bombings', 'NNS', 'O'), ('.', '.', 'O'), ('"', '``', 'O')]


In [53]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
        'word.isalpha()': word.isalpha(),
        'word.isalnum()': word.isalnum(),
        'word.startswith.upper()': word[0].isupper(),
        'word.endswith.s': word.endswith('s'),
        'word.length': len(word),
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
            '-1:word.isalpha()': word1.isalpha(),
            '-1:word.isalnum()': word1.isalnum(),
            '-1:word.startswith.upper()': word1[0].isupper(),
            '-1:word.endswith.s': word1.endswith('s'),
            '-1:word.length': len(word1),
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
            '+1:word.isalpha()': word1.isalpha(),
            '+1:word.isalnum()': word1.isalnum(),
            '+1:word.startswith.upper()': word1[0].isupper(),
            '+1:word.endswith.s': word1.endswith('s'),
            '+1:word.length': len(word1),
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]


In [54]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [58]:

crf = CRF(algorithm = 'l2sgd',
         c2 = 0.1,
         max_iterations = 100,
         all_possible_transitions = False)
crf.fit(X_train, y_train)

CRF(algorithm='l2sgd', all_possible_transitions=False, c2=0.1,
    max_iterations=100)

In [59]:
#Predicting on the test set.
y_pred = crf.predict(X_test)

#### Evaluating the model performance.


In [60]:
f1_score = flat_f1_score(y_test, y_pred, average = 'weighted')
print(f1_score)

0.9701058703069161


In [61]:
flat_f1_score(y_test, y_pred, average = 'weighted')

0.9701058703069161

In [62]:
flat_precision_score(y_test, y_pred, average = 'weighted')

0.9704864959801368

In [63]:
sequence_accuracy_score(y_test, y_pred)

0.6823394495412844

In [64]:
flat_recall_score(y_test, y_pred, average = 'weighted')

0.9708544386578203

In [65]:
flat_accuracy_score(y_test, y_pred)

0.9708544386578203

In [66]:
report = flat_classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

       B-art       0.46      0.11      0.17        57
       B-eve       0.65      0.24      0.35        72
       B-geo       0.83      0.92      0.87      7433
       B-gpe       0.96      0.94      0.95      3161
       B-nat       0.59      0.30      0.39        44
       B-org       0.82      0.74      0.77      4057
       B-per       0.87      0.78      0.83      3392
       B-tim       0.95      0.81      0.88      4019
       I-art       0.27      0.09      0.13        46
       I-eve       0.89      0.11      0.20        73
       I-geo       0.85      0.74      0.79      1480
       I-gpe       0.91      0.65      0.76        48
       I-nat       1.00      0.38      0.56        13
       I-org       0.82      0.82      0.82      3369
       I-per       0.83      0.92      0.88      3459
       I-tim       0.85      0.72      0.78      1333
           O       0.99      1.00      0.99    178199

    accuracy              

## change the input to any sentence, for which you want NER

In [67]:

sentence = "India is going to win the Apple stocks and can get a profit of 2 billion dollars in the next year 2020 with 2kg of apples"  


In [68]:
import nltk
nltk.download('averaged_perceptron_tagger_eng')
tokens = nltk.word_tokenize(sentence)
pos_tags = nltk.pos_tag(tokens)
print(pos_tags)


crf.predict([sent2features(pos_tags)])

[('India', 'NNP'), ('is', 'VBZ'), ('going', 'VBG'), ('to', 'TO'), ('win', 'VB'), ('the', 'DT'), ('Apple', 'NNP'), ('stocks', 'NNS'), ('and', 'CC'), ('can', 'MD'), ('get', 'VB'), ('a', 'DT'), ('profit', 'NN'), ('of', 'IN'), ('2', 'CD'), ('billion', 'CD'), ('dollars', 'NNS'), ('in', 'IN'), ('the', 'DT'), ('next', 'JJ'), ('year', 'NN'), ('2020', 'CD'), ('with', 'IN'), ('2kg', 'CD'), ('of', 'IN'), ('apples', 'NNS')]


[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\pubg3\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


array([['B-geo', 'O', 'O', 'O', 'O', 'O', 'B-org', 'O', 'O', 'O', 'O',
        'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-tim', 'O',
        'O', 'O', 'O']], dtype=object)

In [69]:
import spacy
from spacy import displacy

ner_tags = crf.predict([sent2features(pos_tags)])[0]

nlp = spacy.load("en_core_web_sm")

doc = nlp(sentence)
for token, ner_tag in zip(doc, ner_tags):
    token.ent_type_ = ner_tag

displacy.render(doc, style="ent", jupyter=True)

In [70]:
options = {"compact": True, "bg": "#09a3d5",
           "color": "white", "font": "Source Sans Pro", "fine_grained": True}
displacy.render(doc, style='dep', jupyter=True, options=options)

In [71]:
for a,b in zip(tokens, crf.predict([sent2features(pos_tags)])[0]):
    print(f"{a} -> {b}") 

India -> B-geo
is -> O
going -> O
to -> O
win -> O
the -> O
Apple -> B-org
stocks -> O
and -> O
can -> O
get -> O
a -> O
profit -> O
of -> O
2 -> O
billion -> O
dollars -> O
in -> O
the -> O
next -> O
year -> O
2020 -> B-tim
with -> O
2kg -> O
of -> O
apples -> O
